In [ ]:
# testing gutenberg retrival
#open direct from remote storage

import requests

url = "https://www.gutenberg.org/ebooks/1342.txt.utf-8"

raw= requests.get(url).content.decode('utf-8-sig')

# print(raw)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# LDA preliminary testing

In [ ]:
!pip install git+https://github.com/alexeyegorov/GuidedLDA

  Cloning https://github.com/alexeyegorov/GuidedLDA to /tmp/pip-req-build-tkklurho
  Running command git clone --filter=blob:none --quiet https://github.com/alexeyegorov/GuidedLDA /tmp/pip-req-build-tkklurho
  Resolved https://github.com/alexeyegorov/GuidedLDA to commit 2ccc26e5b53e4c66aa727c1432cba0f270f61f01
  Preparing metadata (setup.py) ... done
  Created wheel for guidedlda: filename=guidedlda-2.0.0.dev27-cp310-cp310-linux_x86_64.whl size=2652648 sha256=79ccfca0780fcecc524574ddb0c2971e21ea8b2358c089c801ec3bdfd82a57bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-3wl368_e/wheels/53/ad/66/0a9804dc9cef8fdb54f3d9b2b2e1d162fbdc80b7151f97085d
Successfully built guidedlda


## Seed Categories

In [ ]:
# Define seed topics
gender_seeds = [['man', 'he', 'him', 'his', 'men', 'boy', 'boys', 'mr', 'sir', 'male', 'son', 'brother', 'father'],
                ['woman', 'she', 'hers', 'girl', 'her', 'girls', 'women', 'feminine', 'mrs', 'miss', 'ms', 'female', 'mother', 'daughter', 'sister']]

racial_seeds = [['white', 'light', 'caucasian', 'european', 'English', 'French', 'pale', 'fair-skinned'],
                ['black', 'dark', 'brown', 'dark-skinned', 'african', 'afro', 'asian', 'indian','oriental', 'chinese', 'korean', 'japanese', 'latino', 'latina', 'hispanic', 'spanish']]

class_seeds = [['wealthy', 'rich', 'affluent', 'elite', 'mansion', 'luxury', 'fortune', 'well-off'],
               ['poor', 'struggling', 'modest', 'middle-class', 'suburban', 'average', 'homeless', 'destitute', 'impoverished', 'working-class']]

seed_topic_list = gender_seeds + racial_seeds + class_seeds

## Data Preprocessing

In [ ]:
import guidedlda
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

punctuation = list(punctuation)
punctuation.append('\"')

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

STOPWORDS = set(stopwords.words('english'))

def extract_adjectives(text):
    # Tokenize and POS tag
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)

    # Return only adjectives excluding stopwords
    return [word for word, tag in pos_tags if tag in ['JJ', 'JJR', 'JJS'] and word.lower() not in STOPWORDS and word.lower() not in punctuation]

# def tokenize(text):
#     # Basic tokenization, you can add stemming, lemmatization and other preprocessing here
#     return [word for word in text.lower().split() if word not in stopwords.words('english')]

# A list of common stopwords. This is a very basic list; consider using libraries like NLTK for a more comprehensive list
# STOPWORDS = set(["a", "an", "the", "and", "or", "of", "to", "is", "in", "it", "that", "on"])

# Assuming `documents` is your list of paragraphs
documents = [raw]

vectorizer = CountVectorizer(analyzer=extract_adjectives, min_df=1, lowercase=True)
X = vectorizer.fit_transform(documents)
word2id = dict((v, idx) for idx, v in enumerate(vectorizer.get_feature_names_out()))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Model with GuidedLDA

In [ ]:
# Seed topics to the model
model = guidedlda.GuidedLDA(n_topics=len(seed_topic_list), n_iter=100, random_state=7, refresh=20)
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        if word in word2id:
          seed_topics[word2id[word]] = t_id

model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

## Display topics to check for biases

In [ ]:
n_top_words = 20
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vectorizer.get_feature_names_out())[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: ” great many last ’ likely new greater sorry several slight strong late civil high bad difficult probable public considerable
Topic 1: least next present less necessary long common delightful fine ill best pleasant fair evident right tolerable painful frequent principal affectionate
Topic 2: “ dear short real Lady natural enough anxious afraid early greatest silent ready it. extraordinary sister respectable disagreeable angry unhappy
Topic 3: good ” possible agreeable impossible handsome second younger easy proper perfect dearest immediate elegant unable half him. grateful comfortable important
Topic 4: first able good former sensible full electronic serious small large superior eldest capable interesting excellent youngest following fortunate false aware
Topic 5: much little young sure happy certain better general whole different amiable usual poor equal particular glad pleased true best proud


# Running LDA on all books - experiments

In [ ]:
from google.colab import drive
import pandas as pd

# drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/nlp-topics-proj/all_books.csv", header='infer')

print(data.head())

FileNotFoundError: ignored

In [ ]:
# import glob
# import pandas as pd
# path = "/content/drive/MyDrive/Colab_Notebooks/nlp-topics-proj/May/new_generate_stories/*.txt"
# # List all txt files in the folder
# txt_files = glob.glob(path)
# # txt_files = ["/content/drive/MyDrive/Colab_Notebooks/nlp-topics-proj/May/new_generate_stories/The Jungle.txt",
# #              "/content/drive/MyDrive/Colab_Notebooks/nlp-topics-proj/May/new_generate_stories/The House of Mirth.txt"]
# generated_data = []
# print(len(txt_files))
# for file_path in txt_files:
#   # print(file_path)
#   file_name = file_path.split('/')[-1]  # Extract file name from the path
#   with open(file_path, 'r') as file:
#     content = file.read()
#     generated_data.append([file_name, content])

# generated_data = pd.DataFrame(generated_data, columns=['Title', 'Content'])

# generated_data.head()


2


,Title,Content
0,The Jungle.txt,It was four o’clock when the ceremony was over...
1,The House of Mirth.txt,Selden paused in surprise. In the dim light of...


##**For GPT-generated stories**

In [ ]:
# # Get all the GPT-generated stories
# # pretty much the same function but slightly modified
# from google.colab import drive
# import pandas as pd

# def evaluate_gpt_stories(row):
#   filtered_content = row["Content"]
#   documents = [filtered_content]
#   vectorizer = CountVectorizer(analyzer=extract_adjectives, min_df=1, lowercase=True)
#   X = vectorizer.fit_transform(documents)
#   word2id = dict((v, idx) for idx, v in enumerate(vectorizer.get_feature_names_out()))

#   # Seed topics to the model
#   model = guidedlda.GuidedLDA(n_topics=len(seed_topic_list), n_iter=100, random_state=7, refresh=20)
#   seed_topics = {}
#   for t_id, st in enumerate(seed_topic_list):
#       for word in st:
#           if word in word2id:
#             seed_topics[word2id[word]] = t_id

#   model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

#   n_top_words = 20
#   topic_word = model.topic_word_
#   current_result = []
#   for i, topic_dist in enumerate(topic_word):
#       topic_words = np.array(vectorizer.get_feature_names_out())[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#       print('Topic {}: {}'.format(i, ' '.join(topic_words)))
#       current_result.append('{}'.format(' '.join(topic_words)))

#   # Your custom function here, which returns 7 values

#   result = (current_result[0], current_result[1], current_result[2], current_result[3], current_result[4], current_result[5])
#   return result

# generated_data[['Male', 'Female', 'White', 'Non-white', 'Wealthy', 'Non-wealthy']] = generated_data.apply(evaluate_gpt_stories, axis=1, result_type='expand')

Topic 0: lush silent hidden overhear fairy white big colorful keen impeccable humanitarian high astonished grand good golden free first familiar life-changing
Topic 1: social youngest distant new next polished practiced keen remarkable trivial big different golden humanitarian high hidden grand good astonished authentic
Topic 2: loyal unfolding middle-aged traditional societal emerald golden youngest first impeccable humanitarian high hidden grand good free fairy familiar leap esteemed
Topic 3: wealthy quiet ambitious stark impeccable free first familiar secret life-changing different astonished authentic distant good keen humanitarian high hidden grand
Topic 4: humanitarian deep supportive steady good esteemed perfect youngest free keen impeccable high hidden grand golden fairy first familiar life-changing emerald
Topic 5: leap young modest colorful high palpable grand free keen impeccable humanitarian hidden good golden youngest life-changing familiar fairy esteemed emerald
Topic 0: 

In [ ]:
# url = "https://www.gutenberg.org/ebooks/1342.txt.utf-8"


def get_book(url):
  return requests.get(url).content.decode('utf-8-sig')

def evaluate_book(row):
    current_url = row['URL']
    documents = [get_book(current_url)]
    vectorizer = CountVectorizer(analyzer=extract_adjectives, min_df=1, lowercase=True)
    X = vectorizer.fit_transform(documents)
    word2id = dict((v, idx) for idx, v in enumerate(vectorizer.get_feature_names_out()))

    # Seed topics to the model
    model = guidedlda.GuidedLDA(n_topics=len(seed_topic_list), n_iter=100, random_state=7, refresh=20)
    seed_topics = {}
    for t_id, st in enumerate(seed_topic_list):
        for word in st:
            if word in word2id:
              seed_topics[word2id[word]] = t_id

    model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

    n_top_words = 20
    topic_word = model.topic_word_
    current_result = []
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vectorizer.get_feature_names_out())[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        print('Topic {}: {}'.format(i, ' '.join(topic_words)))
        current_result.append('{}'.format(' '.join(topic_words)))

    # Your custom function here, which returns 7 values

    result = (current_result[0], current_result[1], current_result[2], current_result[3], current_result[4], current_result[5])
    return result

# Apply the function to each row and store the result in 7 new columns
data[['Male', 'Female', 'White', 'Non-white', 'Wealthy', 'Non-wealthy']] = data.apply(evaluate_book, axis=1, result_type='expand')

Topic 0: ” great many last ’ likely new greater sorry several slight strong late civil high bad difficult probable public considerable
Topic 1: least next present less necessary long common delightful fine ill best pleasant fair evident right tolerable painful frequent principal affectionate
Topic 2: “ dear short real Lady natural enough anxious afraid early greatest silent ready it. extraordinary sister respectable disagreeable angry unhappy
Topic 3: good ” possible agreeable impossible handsome second younger easy proper perfect dearest immediate elegant unable half him. grateful comfortable important
Topic 4: first able good former sensible full electronic serious small large superior eldest capable interesting excellent youngest following fortunate false aware
Topic 5: much little young sure happy certain better general whole different amiable usual poor equal particular glad pleased true best proud
Topic 0: great dark beautiful afraid secret — possible green happy single dismal na

In [ ]:
print(data.head())
data.to_csv('lda_results_adj_only.csv', index=False)

                                Title             Author  \
0                 Pride and Prejudice        Jane Austen   
1                  Great Expectations    Charles Dickens   
2                           Jane Eyre   Charlotte Bronte   
3                   Wuthering Heights       Emily Bronte   
4  The Adventures of Huckleberry Finn         Mark Twain   

                                               URL                    Bias  \
0  https://www.gutenberg.org/ebooks/1342.txt.utf-8             gender bias   
1  https://www.gutenberg.org/ebooks/1400.txt.utf-8              class bias   
2  https://www.gutenberg.org/ebooks/1260.txt.utf-8   gender and class bias   
3   https://www.gutenberg.org/ebooks/768.txt.utf-8              class bias   
4    https://www.gutenberg.org/ebooks/76.txt.utf-8             racial bias   

                                         Description  \
0  This novel illustrates gender bias through its...   
1  Dickens critiques class bias by following Pip'...   
2 

In [ ]:
generated_data.to_csv('lda_results_generated_stories_adj_only.csv', index=False)
generated_data.head()

,Title,Content,Male,Female,White,Non-white,Wealthy,Non-wealthy
0,The Jungle.txt,It was four o’clock when the ceremony was over...,lush silent hidden overhear fairy white big co...,social youngest distant new next polished prac...,loyal unfolding middle-aged traditional societ...,wealthy quiet ambitious stark impeccable free ...,humanitarian deep supportive steady good estee...,leap young modest colorful high palpable grand...
1,The House of Mirth.txt,Selden paused in surprise. In the dim light of...,young wildest Eldorian age-old ancient closer ...,old ancient ready mutual wildest true seasoned...,young vast age-old strange deeper lay lost Eld...,hidden 'This closer final quiet promised labyr...,unspoken forgotten library young lost Eldorian...,Eldorian relentless past new lost unknown anci...
